<a href="https://colab.research.google.com/github/drshahizan/Python_EDA/blob/main/assignment/ass5/bdm/TheBoys/TheBoys_Assignment_5a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TheBoys - Assignment 5a
1. Pang Chern Hong | MCS231006
2. Nian Cong    | MCS231007
3. Wu Jiaming    | MCS221033
4. Liu Kaiyuan   | MCS231020

In [1]:
pip install geotext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.4 MB/s eta 0:00:00


In [2]:
pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geotext
import collections
import string, sys, os, re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from IPython.display import Image
from functools import reduce
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/brooklyn_sales_map.csv")
df

<ipython-input-5-e11e96fdfbd0>:1: DtypeWarning: Columns (40,41,43,45,46,47,86) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/brooklyn_sales_map.csv")


,Unnamed: 0,borough,neighborhood,building_class_category,tax_class,block,lot,easement,building_class,address,...,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,MAPPLUTO_F,SHAPE_Leng,SHAPE_Area
0,1,3,DOWNTOWN-METROTECH,28 COMMERCIAL CONDOS,4,140,1001,NaN,R5,330 JAY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3,DOWNTOWN-FULTON FERRY,29 COMMERCIAL GARAGES,4,54,1,NaN,G7,85 JAY STREET,...,NaN,3.000540e+09,12/06/2002,1.0,NaN,NaN,17V1.1,0.0,1559.889144,140131.577176
2,3,3,BROOKLYN HEIGHTS,21 OFFICE BUILDINGS,4,204,1,NaN,O6,29 COLUMBIA HEIGHTS,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,890.718521,34656.447240
3,4,3,MILL BASIN,22 STORE BUILDINGS,4,8470,55,NaN,K6,5120 AVENUE U,...,NaN,0.000000e+00,NaN,1.0,1.0,1.0,17V1.1,0.0,3729.786857,797554.847834
4,5,3,BROOKLYN HEIGHTS,26 OTHER HOTELS,4,230,1,NaN,H8,21 CLARK STREET,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,620.761169,21360.147631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390878,390879,3,WYCKOFF HEIGHTS,31 COMMERCIAL VACANT LAND,1,3337,19,NaN,C0,364 LINDEN STREET,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,244.422014,2501.088819
390879,390880,3,WYCKOFF HEIGHTS,31 COMMERCIAL VACANT LAND,1,3337,19,NaN,C0,364 LINDEN STREET,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,244.422014,2501.088819
390880,390881,3,WYCKOFF HEIGHTS,41 TAX CLASS 4 - OTHER,4,3280,50,NaN,Z9,383 HARMAN STREET,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,231.409296,2795.673780
390881,390882,3,3004,04 TAX CLASS 1 CONDOS,NaN,6886,1107,NaN,NaN,14 BAY 47TH STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#EDA

1,Check for Missing Values:

In [6]:
import pandas as pd
missing_values = df.isnull().sum()
print(missing_values)

Unnamed: 0                      0
borough                         0
neighborhood                    0
building_class_category        83
tax_class                    6934
                            ...  
PFIRM15_FL                 363110
Version                     87155
MAPPLUTO_F                  87155
SHAPE_Leng                  87155
SHAPE_Area                  87155
Length: 111, dtype: int64


2,Understand Data Types:

In [7]:
data_types = df.dtypes
print(data_types)

Unnamed: 0                   int64
borough                      int64
neighborhood                object
building_class_category     object
tax_class                   object
                            ...   
PFIRM15_FL                 float64
Version                     object
MAPPLUTO_F                 float64
SHAPE_Leng                 float64
SHAPE_Area                 float64
Length: 111, dtype: object


3,Grasp the Structure of the Dataset:

In [8]:
head = df.head()
print(head)

   Unnamed: 0  borough           neighborhood building_class_category  \
0           1        3     DOWNTOWN-METROTECH   28  COMMERCIAL CONDOS   
1           2        3  DOWNTOWN-FULTON FERRY  29  COMMERCIAL GARAGES   
2           3        3       BROOKLYN HEIGHTS    21  OFFICE BUILDINGS   
3           4        3             MILL BASIN     22  STORE BUILDINGS   
4           5        3       BROOKLYN HEIGHTS         26 OTHER HOTELS   

  tax_class  block   lot  easement building_class              address  ...  \
0         4    140  1001       NaN             R5       330 JAY STREET  ...   
1         4     54     1       NaN             G7        85 JAY STREET  ...   
2         4    204     1       NaN             O6  29 COLUMBIA HEIGHTS  ...   
3         4   8470    55       NaN             K6        5120 AVENUE U  ...   
4         4    230     1       NaN             H8      21 CLARK STREET  ...   

  EDesigNum        APPBBL     APPDate  PLUTOMapID  FIRM07_FLA  PFIRM15_FL  \
0       N

4,Basic Statistics:

In [9]:
statistics = df.describe()
print(statistics)

          Unnamed: 0   borough          block            lot  easement  \
count  390883.000000  390883.0  390883.000000  390883.000000       0.0   
mean   195442.000000       3.0    4275.651169     272.509986       NaN   
std    112838.346973       0.0    2642.765332     484.658837       NaN   
min         1.000000       3.0       1.000000       1.000000       NaN   
25%     97721.500000       3.0    1786.000000      21.000000       NaN   
50%    195442.000000       3.0    4305.000000      45.000000       NaN   
75%    293162.500000       3.0    6585.000000      94.000000       NaN   
max    390883.000000       3.0    8955.000000    9133.000000       NaN   

            zip_code  residential_units  commercial_units    total_units  \
count  390883.000000      390883.000000     390883.000000  390883.000000   
mean    11208.429087           2.486138          0.174142       2.673227   
std       361.207836          48.200178          1.653362      48.233343   
min         0.000000         

#Feature Engineering with Automated Tools

1,Featuretools

In [10]:
pip install featuretools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.2/619.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.4/754.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: holidays
    Found existing installation: holidays 0.36
    Uninstalling holidays-0.36:
      Successfully uninstalled holidays-0.36


In [11]:
import featuretools as ft

In [12]:
data=df

In [23]:
es = ft.demo.load_mock_customer(return_entityset=True)
es

Entityset: transactions
  DataFrames:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 3]
    sessions [Rows: 35, Columns: 5]
    customers [Rows: 5, Columns: 5]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id

In [28]:
data["index"] = data["sale_date"]
data = data.set_index("index")

In [36]:
sale_price_df= pd.DataFrame(data["sale_price"],data["sale_date"])
sale_price_df.sample(5)

,sale_price
sale_date,
2011-01-07,0.0
2013-12-12,0.0
2010-08-12,0.0
2014-04-14,0.0
2003-10-16,0.0


In [37]:
land_sqft_df = pd.DataFrame(data["land_sqft"],data["sale_date"])
land_sqft_df.sample(5)

,land_sqft
sale_date,
2005-12-06,0.0
2013-12-30,1900.0
2016-09-28,0.0
2013-08-21,0.0
2009-07-27,1600.0


In [45]:
LotArea_df=pd.DataFrame(data["LotArea"],data["sale_date"],data["zip_code"])
LotArea_df.sample(5)

KeyboardInterrupt: ignored

In [41]:
from featuretools.primitives import Mean


In [78]:
child_df = pd.DataFrame({"index" :df["Unnamed: 0"], "LotArea": df["LotArea"],
                                "SchoolDist": df["SchoolDist"],
                                "sale_date": df["sale_date"],
                                "sale_price": df["sale_price"]})

In [79]:
es.add_dataframe(dataframe_name="lotarea_dist",
                 index="index",
                 time_index="sale_date",
                 dataframe=child_df)

Entityset: result
  DataFrames:
    sale_price [Rows: 390883, Columns: 5]
    lotarea_dist [Rows: 390883, Columns: 5]
  Relationships:
    No relationships

In [74]:
transactions_df = pd.DataFrame({"index": df["Unnamed: 0"].astype(str),
                                "land_sqft": df["land_sqft"],
                                "zip_code": df["zip_code"],
                                "sale_date": df["sale_date"],
                                "sale_price": df["sale_price"]})

In [75]:
es = ft.EntitySet("result")

In [76]:
es.add_dataframe(dataframe_name="sale_price",
                 index="index",
                 time_index="sale_date",
                 dataframe=transactions_df)

Entityset: result
  DataFrames:
    sale_price [Rows: 390883, Columns: 5]
  Relationships:
    No relationships

In [82]:
new_relationship = ft.Relationship(entityset=es,parent_dataframe_name='lotarea_dist',parent_column_name='index',
                    child_dataframe_name='sale_price',child_column_name='sale_price')

In [83]:
es = es.add_relationship(relationship=new_relationship)

/usr/local/lib/python3.10/dist-packages/featuretools/entityset/entityset.py:414: UserWarning: Logical type Double for child column sale_price does not match parent column index logical type Integer. Changing child logical type to match parent.
  warnings.warn(


In [84]:
feature_matrix, features = ft.dfs( entityset=es,
    target_dataframe_name="sale_price",
    max_depth=2,
)
feature_matrix.head(5)

,land_sqft,zip_code,sale_price,DAY(sale_date),MONTH(sale_date),WEEKDAY(sale_date),YEAR(sale_date),lotarea_dist.LotArea,lotarea_dist.SchoolDist,lotarea_dist.sale_price,...,lotarea_dist.SKEW(sale_price.land_sqft),lotarea_dist.SKEW(sale_price.zip_code),lotarea_dist.STD(sale_price.land_sqft),lotarea_dist.STD(sale_price.zip_code),lotarea_dist.SUM(sale_price.land_sqft),lotarea_dist.SUM(sale_price.zip_code),lotarea_dist.DAY(sale_date),lotarea_dist.MONTH(sale_date),lotarea_dist.WEEKDAY(sale_date),lotarea_dist.YEAR(sale_date)
index,,,,,,,,,,,,,,,,,,,,,
58000,5000.0,11222,715809,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
58001,5000.0,11222,715809,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
58002,3890.0,11222,715809,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
66546,5000.0,11222,655490,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
66547,5000.0,11222,655490,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


Understanding feature output

In [90]:
feature = features[18]
feature

<Feature: lotarea_dist.SKEW(sale_price.zip_code)>

In [96]:
feature_matrix

,land_sqft,zip_code,sale_price,DAY(sale_date),MONTH(sale_date),WEEKDAY(sale_date),YEAR(sale_date),lotarea_dist.LotArea,lotarea_dist.SchoolDist,lotarea_dist.sale_price,...,lotarea_dist.SKEW(sale_price.land_sqft),lotarea_dist.SKEW(sale_price.zip_code),lotarea_dist.STD(sale_price.land_sqft),lotarea_dist.STD(sale_price.zip_code),lotarea_dist.SUM(sale_price.land_sqft),lotarea_dist.SUM(sale_price.zip_code),lotarea_dist.DAY(sale_date),lotarea_dist.MONTH(sale_date),lotarea_dist.WEEKDAY(sale_date),lotarea_dist.YEAR(sale_date)
index,,,,,,,,,,,,,,,,,,,,,
58000,5000.0,11222,715809,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
58001,5000.0,11222,715809,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
58002,3890.0,11222,715809,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
66546,5000.0,11222,655490,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
66547,5000.0,11222,655490,1,1,2,2003,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255572,34059.0,11225,0,31,12,6,2017,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
255573,37560.0,11225,0,31,12,6,2017,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
255574,100.0,11225,0,31,12,6,2017,<NA>,<NA>,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
